In [1]:
import requests
import json
import pandas as pd

https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento?uf=DF&pagina=83

In [2]:
## Trazendo as informações da api com o método get

def fetch_data(filters={}):

  resultado = []
  count = 0

  while(True):

    url = (f'https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento?uf=DF&pagina={count}')
    response = requests.get(url, params=filters)

    if response.status_code == 200:
      data = response.json()
      if data and data['content']: # Check if data and its content are not None or empty
          print(f"Adicionando dados na iteração nº: {count}")
          resultado.extend(data['content']) # Use extend to add elements from the list
      else:
          print("Parando: Nenhum conteúdo na página ou erro no endpoint da api")
          break # Exit the loop if no content is returned

    else:
      print(f"Erro na iteração nº: {count} com status code: {response.status_code}")
      break

    if count == 15:
       break
    
    count+= 1

  return resultado

In [3]:
dados = fetch_data()

Adicionando dados na iteração nº: 0
Erro na iteração nº: 1 com status code: 429


In [4]:
df = pd.DataFrame(dados)

In [5]:
df[['idUnico','nome','subTipos', 'tipos']]

,idUnico,nome,subTipos,tipos
0,50379.53-54,DL - 304/2024 - Contratação de instituição par...,"[{'id': 4, 'descricao': 'Acessos Terrestres', ...","[{'id': 25, 'descricao': 'Rodovia', 'idEixo': 3}]"
1,42724.53-27,Escola Classe Crixá São Sebastião,"[{'id': 84, 'descricao': 'Educação', 'idTipo':...","[{'id': 46, 'descricao': 'Educação', 'idEixo':..."
2,19970.53-78,Reajuste do Contrato 45/2021 - Contrução do Ce...,"[{'id': 59, 'descricao': 'Obras em Imóveis de ...","[{'id': 1, 'descricao': 'Segurança Pública', '..."
3,24797.53-15,Implantação de Passarelas nas Estradas Parque ...,"[{'id': 57, 'descricao': 'Obra de Arte Especia...","[{'id': 24, 'descricao': 'Infraestrutura Urban..."
4,24822.53-70,"obra de construção da Cabine de Medição, loca...","[{'id': 95, 'descricao': 'Subestação', 'idTipo...","[{'id': 31, 'descricao': 'Energia', 'idEixo': ..."
...,...,...,...,...
155,8509.53-20,AET,"[{'id': 4, 'descricao': 'Acessos Terrestres', ...","[{'id': 25, 'descricao': 'Rodovia', 'idEixo': 3}]"
156,8513.53-14,AET,"[{'id': 4, 'descricao': 'Acessos Terrestres', ...","[{'id': 25, 'descricao': 'Rodovia', 'idEixo': 3}]"
157,74143.53-04,PAC 2 - CRECHE/PRÉ-ESCOLA 024,"[{'id': 36, 'descricao': 'Dragagem, Derrocamen...","[{'id': 22, 'descricao': 'Infraestrutura Hídri..."
158,8726.53-42,contratação de empresa especializada em elabor...,"[{'id': 76, 'descricao': 'Usina', 'idTipo': 31...","[{'id': 31, 'descricao': 'Energia', 'idEixo': ..."


In [6]:
df['tomadorNome'] = df['tomadores'].apply(lambda x: x[0]['nome'] if len(x) > 0 else None)
df['tomadorCodigo'] = df['tomadores'].apply(lambda x: x[0]['codigo'] if len(x) > 0 else None)

In [7]:
df['executorNome'] = df['executores'].apply(lambda x: x[0]['nome'] if len(x) > 0 else None)
df['executorCodigo'] = df['executores'].apply(lambda x: x[0]['codigo'] if len(x) > 0 else None)

In [8]:
df['repassadorNome'] = df['repassadores'].apply(lambda x: x[0]['nome'] if len(x) > 0 else None)
df['repassadorCodigo'] = df['repassadores'].apply(lambda x: x[0]['codigo'] if len(x) > 0 else None)

In [9]:
df['origemFontesDeRecurso'] = df['fontesDeRecurso'].apply(lambda x: x[0]['origem'] if len(x) > 0 else None)
df['valorInvestimentoPrevisto'] = df['fontesDeRecurso'].apply(lambda x: x[0]['valorInvestimentoPrevisto'] if len(x) > 0 else None)

In [10]:
df_principal = df.drop(columns=['tomadores','executores','repassadores','fontesDeRecurso'])

In [11]:
# Listas para armazenar os dados extraídos
eixosData = []
tiposData = []
subtiposData = []

# Listas para as tabelas de ligação
operacaoEixoRel = []
operacaoTipoRel = []
operacaoSubtipoRel = []

In [12]:
# Iterar sobre o DataFrame original para extrair os dados aninhados
for index, row in df_principal.iterrows():
    id_op = row['idUnico']

    for eixo in row['eixos']:
        eixosData.append(eixo)
        operacaoEixoRel.append({'idUnico': id_op, 'id_eixo': eixo['id']})

    for tipo in row['tipos']:
        tiposData.append(tipo)
        operacaoTipoRel.append({'idUnico': id_op, 'id_tipo': tipo['id']})

    for subtipo in row['subTipos']:
        subtiposData.append(subtipo)
        operacaoSubtipoRel.append({'idUnico': id_op, 'id_subtipo': subtipo['id']})

# --- Criar os novos DataFrames (Tabelas de Dimensão) ---

# Tabela de Eixos
df_eixos = pd.DataFrame(eixosData).drop_duplicates().reset_index(drop=True)

# Tabela de Tipos
df_tipos = pd.DataFrame(tiposData).drop_duplicates().reset_index(drop=True)

# Tabela de SubTipos
df_subtipos = pd.DataFrame(subtiposData).drop_duplicates().reset_index(drop=True)

In [13]:
df_principal_final = df_principal.drop(columns=['tipos','subTipos','eixos'])

In [ ]:
df_operacaoEixoRel = pd.DataFrame(operacaoEixoRel).drop_duplicates().reset_index(drop=True)
df_operacaoTipoRel = pd.DataFrame(operacaoTipoRel).drop_duplicates().reset_index(drop=True)
df_operacaoSubtipoRel = pd.DataFrame(operacaoSubtipoRel).drop_duplicates().reset_index(drop=True)

In [25]:
df_operacaoEixoRel

,idUnico,id_eixo
0,50379.53-54,3
1,42724.53-27,4
2,19970.53-78,1
3,24797.53-15,3
4,24822.53-70,3
...,...,...
154,8509.53-20,3
155,8513.53-14,3
156,74143.53-04,3
157,8726.53-42,3


In [14]:
df_principal_final

,idUnico,nome,cep,endereco,descricao,funcaoSocial,metaGlobal,dataInicialPrevista,dataFinalPrevista,dataInicialEfetiva,...,isModeladaPorBim,dataSituacao,tomadorNome,tomadorCodigo,executorNome,executorCodigo,repassadorNome,repassadorCodigo,origemFontesDeRecurso,valorInvestimentoPrevisto
0,50379.53-54,DL - 304/2024 - Contratação de instituição par...,None,None,Contratação de instituição para execução de se...,Ampliação da capacidade de trafego visando a m...,Projetos Básicos e Executivos de Engenharia,2024-12-20,2027-12-05,None,...,False,2024-12-20,None,NaN,DEPARTAMENTO NACIONAL DE INFRAESTRUTURA DE TRA...,5.484400e+04,None,NaN,Federal,44463443.00
1,42724.53-27,Escola Classe Crixá São Sebastião,None,None,"Construção de Escola em Tempo Integral, Escola...",A construção da nova escola beneficiará 977 es...,"Construção de Escola em Tempo Integral, Escola...",2024-09-02,2028-09-02,None,...,False,2025-09-05,None,NaN,SECRETARIA DE ESTADO DE EDUCACAO DO DISTRITO F...,3.946760e+11,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCAÇÃO,2.530000e+02,Federal,12319519.51
2,19970.53-78,Reajuste do Contrato 45/2021 - Contrução do Ce...,70.602-600,"SAIS Área Especial 3, Setor Policial Sul",Reajuste do Contrato 45/2021 - Construção do C...,Contribuir para a melhor formação dos bombeiro...,Construção de um novo centro de formação e de ...,2021-09-14,2024-08-28,None,...,False,2023-02-06,None,NaN,CORPO DE BOMBEIROS MILITAR DO DISTRITO FEDERAL,8.977914e+12,CORPO DE BOMBEIROS MILITAR DO DISTRITO FEDERAL,8.977914e+12,Federal,1177429.91
3,24797.53-15,Implantação de Passarelas nas Estradas Parque ...,None,None,Implantação de passarelas de estrutura mista n...,"Pedestres, no geral, demanda das ocupações lin...",Implantação de passarelas de estrutura mista n...,2023-08-30,2028-08-30,None,...,False,2023-08-28,None,NaN,DEPARTAMENTO DE ESTRADAS DE RODAGEM DO DISTRIT...,7.053200e+10,MINISTÉRIO DAS CIDADES,3.087980e+05,Federal,10800000.00
4,24822.53-70,"obra de construção da Cabine de Medição, loca...",None,None,"obra de construção da Cabine de Medição, loca...",A demanda de carga elétrica do Campus Darcy Ri...,A demanda de carga elétrica do Campus Darcy Ri...,2023-09-14,2024-03-14,None,...,False,2023-08-29,None,NaN,FUNDACAO UNIVERSIDADE DE BRASILIA,2.627100e+04,FUNDACAO UNIVERSIDADE DE BRASILIA,2.627100e+04,Federal,928139.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,8509.53-20,AET,None,None,Serviços de apoio técnico à gestão de processa...,Preservação do pavimento das rodovias e conseq...,Preservação do pavimento das rodovias e conseq...,2019-04-02,2022-09-29,None,...,False,2022-09-28,None,NaN,DEPARTAMENTO NACIONAL DE INFRAESTRUTURA DE TRA...,5.484400e+04,None,NaN,Federal,12353524.35
156,8513.53-14,AET,None,None,Execução dos serviços de apoio técnico à verif...,Promover a verificação de peso e dimensões de ...,Promover a verificação de peso e dimensões de ...,2019-04-02,2023-09-29,None,...,False,2022-09-28,None,NaN,DEPARTAMENTO NACIONAL DE INFRAESTRUTURA DE TRA...,5.484400e+04,None,NaN,Federal,7537303.03
157,74143.53-04,PAC 2 - CRECHE/PRÉ-ESCOLA 024,None,None,PAC 2 - CRECHE/PRÉ-ESCOLA 024 Tipo de Obra,PAC 2 - CRECHE/PRÉ-ESCOLA 024,Escola de Educação Infantil Tipo B,2013-04-05,2014-05-10,None,...,False,2025-07-25,None,NaN,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCAÇÃO,2.530000e+02,None,NaN,Federal,0.01
158,8726.53-42,contratação de empresa especializada em elabor...,None,None,Contratação de empresa especializada na (i) el...,Justifica-se pela necessidade de modernizar e ...,Elaboração e execução de projetos de usinas fo...,2022-10-19,2023-10-24,None,...,False,2022-09-29,FUNDACAO UNIVERSIDADE DE BRASILIA,26271.0,FUNDACAO UNIVERSIDADE DE BRASILIA,2.627100e+04,None,NaN,Federal,2902400.00


Data Frames:

```
df_principal_final
df_eixos
df_tipos
df_subtipos
df_operacaoEixoRel
df_operacaoTipoRel
df_operacaoSubTipoRel
```



